<a href="https://colab.research.google.com/github/Rajasettu/hot-star/blob/main/Copy_of_respiratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obss/BIOBSS/blob/main/examples/respiratory_analysis.ipynb)

In [ ]:
https://github.com/Rajasettu/hot-star.git

SyntaxError: invalid syntax (ipython-input-2584172791.py, line 1)

__BIOBSS - Respiratory Analysis__

_This notebook includes guidelines to help using BIOBSS for respiratory signal extraction and respiratory rate estimation from PPG signal._

__To run this notebook in Colab:__

In [ ]:
%%bash
git clone https://github.com/obss/biobss.git
cd biobss
pip install .

In [ ]:
!pip install -U matplotlib
import shutil
from biobss import FIXTURES_ROOT
shutil.move("/content/biobss/sample_data",FIXTURES_ROOT.parent)

__Import required packages:__

In [ ]:
import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Table of Contents
1.[PPG Sample Data](#sampledata)<br>
2.[Preprocessing](#resp_pre)<br>
3.[Extraction of Respiratory Signals](#resp_sig)<br>
4.[Plotting](#resp_plot)<br>
5.[Respiratory Signal Filtering](#filt_resp)<br>
6.[Calculation of Respiratory Quality Indices](#resp_rqi)<br>
7.[Respiratory Rate Estimation](#resp_rr)<br>
8.[Fusion of Respiratory Rates](#resp_fusion)<br>

In this tutorial notebook, PPG sample data is used for respiratory analysis. Note that, the same procedures can be applied on ECG sample data.

### __PPG Sample Data__
<a id="sampledata"></a>

PPG sample data is provided as a csv file in BIOBSS\sample data. The data file contains one PPG segment of 60-seconds length. The sampling rate of the signal is 64 Hz.

In [ ]:
#Load the sample data
data, info = biobss.utils.load_sample_data(data_type='PPG_long')
sig = np.asarray(data['PPG'])
fs = info['sampling_rate']
L = info['signal_length']

### __Preprocessing__
<a id="resp_pre"></a>

For respiratory rate estimation, the PPG signal should be filtered before extracting respiratory signal(s). The __resp_preprocess__ module includes functions with predefined filtering parameters to eliminate very high and very low frequencies.

In [ ]:
#ELiminate very low frequencies. 5th order Butterworth highpass filter with 0.0665 Hz (4 bpm) cutoff frequency.
filt_sig=biobss.resptools.elim_vlf(sig, sampling_rate=fs)
#ELiminate very low frequencies. 5th order Butterworth lowpass filter with 0.5833 (35 bpm) cutoff frequency.
filt_sig=biobss.resptools.elim_vhf(sig, sampling_rate=fs)

For the extraction of respiratory signals from modulations in the PPG signal, peak and trough locations are needed. To be able to calculate the modulations, number of peaks and troughs should match and the locations should be in order. Thus, peak control procedure should also be applied by setting __correct_peaks__ as True.

In [ ]:
#Peak detection
info=biobss.ppgtools.ppg_detectpeaks(sig=sig, sampling_rate=fs, method='peakdet', delta=0.01, correct_peaks=True)
locs_peaks=info['Peak_locs']
peaks=sig[locs_peaks]
locs_onsets=info['Trough_locs']
onsets=sig[locs_onsets]

### __Extraction of Respiratory Signals__
<a id="resp_sig"></a>

The ___extract_resp_sig___ function can be used to extract respiratory signals based on three modulation types which are Amplitude Modulation (AM), Frequency Modulation (FM) and Baseline Wander (BW). The function returns a dictionary including amplitude values and corresponding sample numbers of respiratory signals for the selected modulation types.

In [ ]:
#Extract the respiratory signal related to 'AM','FM' and'BW' from the PPG signal.

info=biobss.resptools.extract_resp_sig(sig=sig, peaks_locs=locs_peaks, troughs_locs=locs_onsets, sampling_rate=fs,mod_type=['AM','FM','BW'],resampling_rate=10)

y_am=info['am_y']
x_am=info['am_x']

y_fm=info['fm_y']
x_fm=info['fm_x']

y_bw=info['bw_y']
x_bw=info['bw_x']

#### __Plotting__
<a id="resp_plot"></a>

BIOBSS provides plotting functions specific to each signal type. In order to plot respiratory signals, ___plot_resp___ function can be used. The signals and peaks should be provided as dictionaries, and the keys should be selected properly as shown below. The plots can be generated either using __Matplotlib__ or __Plotly__.

In [ ]:
#Generate inputs as dictionaries
signals = {'AM': {'x': x_am, 'y':y_am}, 'FM': {'x': x_fm, 'y':y_fm}, 'BW': {'x': x_bw, 'y':y_bw}}

In [ ]:
#Plot respiratory signals using Matplotlib
biobss.resptools.plot_resp(signals=signals, sampling_rate=fs, show_peaks=False, figsize=(10,5))

In [ ]:
#Plot respiratory signals using Plotly
biobss.resptools.plot_resp(signals=signals, sampling_rate=fs, method='plotly', show_peaks=False, width=800, height=400)

### __Respiratory Signal Filtering__
<a id="filt_resp"></a>

If required, ___filter_resp_sig___ function can be used to filter respiratory signal using __rsp_clean__ function from __Neurokit2__ package.

In [ ]:
#Filter respiratory signals
info=biobss.resptools.filter_resp_sig(resampling_rate=10, am_sig=y_am, am_x=x_am, fm_sig=y_fm, fm_x=x_fm, bw_sig=y_bw, bw_x=x_bw)

y_am=info['am_sig']
x_am=info['am_x']

y_fm=info['fm_sig']
x_fm=info['fm_x']

y_bw=info['bw_sig']
x_bw=info['bw_x']

### __Calculation of Respiratory Quality Indices__
<a id="resp_rqi"></a>

Quality of respiratory signals extracted from modulations in PPG signal depends on quality of PPG signal, other artifacts that may be in the same frequency range with the respiratory activity and demographic features of the subject. Thus, respiration rates may differ in accuracy for each respiratory signal. Respiratory quality indices (RQI) can be defined as a measure of quality of respiratory signals. In the literature, there are several respiratory quality indices defined for this purpose. BIOBSS library provides the function ___calc_rqi___ to calculate 'Autocorrelation' and 'Hjorth' respiratory quality indices.

In [ ]:
#Calculate respiratory quality indices. Note that, this function should be applied for each modulation type seperately.
#The function returns a dictionary of respiratory quality indices.
rqi_am=biobss.resptools.calc_rqi(y_am,resampling_rate=10)
rqi_fm=biobss.resptools.calc_rqi(y_fm,resampling_rate=10)
rqi_bw=biobss.resptools.calc_rqi(y_bw,resampling_rate=10)

### __Respiratory Rate Estimation__
<a id="resp_rr"></a>

Respiratory rate can be estimated from respiratory signals using various methods. The ___estimate_rr___ function provides two options which are 'peakdet' and 'xcorr' methods. The 'peakdet' method calculates respiratory rate by detecting peaks of respiratory signal and calculating the peak to peak intervals. The method __xcorr__ is imported from __Neurokit2__ library. It calculates cross-correlations between the changes in respiration with a bank of sinusoids of different frequencies to identify the principal frequency of oscillation.

In [ ]:
#Estimate respiratory rate using 'peakdet' method. Note that, this function should be applied for each modulation type seperately.
rr_am=biobss.resptools.estimate_rr(y_am,10,method='xcorr')
rr_fm=biobss.resptools.estimate_rr(y_fm,10,method='xcorr')
rr_bw=biobss.resptools.estimate_rr(y_bw,10,method='peakdet')


### __Fusion of Respiratory Rates__
<a id="resp_fusion"></a>

Estimated respiratory rates can be fused into a single value using ___fuse_rr___ function. The function provides two options for fusion method which are 'SmartFusion' and 'QualityFusion'.

'SmartFusion' calculates the average of respiratory rates considering the standart deviation of the estimates.

In [ ]:
#Fuse the respiratory rates using 'SmartFusion' method.
rr_fused=biobss.resptools.fuse_rr(rr_est=[rr_am, rr_fm, rr_bw], fusion_method='SmartFusion')

'QualityFusion' takes respiratory quality indices into account. This time, rqi values should also be provided and the elements of rqi and rr_est arrays should be in the same order.

In [ ]:
#Fuse the respiratory rates using 'QualityFusion' method.
#RQIs (array) should be provided as a keyworded argument.
#The order of elements in the array should match with the order RRs of different modulation types.
rqi=[rqi_am['hjorth'],rqi_fm['hjorth'],rqi_bw['hjorth']]
rr_fused=biobss.resptools.fuse_rr(rr_est=[rr_am, rr_fm, rr_bw], rqi=rqi, fusion_method='QualityFusion')